##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with the Gemini API: Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/get-started/python"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/get-started/python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/get-started/python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This quickstart demonstrates how to use the Python SDK for the Gemini API, which gives you access to Google's Gemini large language models. In this quickstart, you will learn how to:

1. Set up your development environment and API access to use Gemini.
2. Generate text responses from text inputs.
3. Generate text responses from multimodal inputs (text and images).
4. Use Gemini for multi-turn conversations (chat).
5. Use embeddings for large language models.

## Prerequisites

You can run this quickstart in [Google Colab](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/get-started/python.ipynb), which runs this notebook directly in the browser and does not require additional environment configuration.

Alternatively, to complete this quickstart locally, ensure that your development environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.

## Setup

### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
 %cd drive
! ls

/content/drive
MyDrive


In [3]:
 %cd MyDrive/FinRAG
! ls

/content/drive/MyDrive/FinRAG
 cache				  FinRAD_preprocess_gemini_colab.ipynb	 output
'Copy of FinRAG Poster.gslides'   generator				 retriever_external
 dataset			  generator_train_test.ipynb		 retriever_internal


In [ ]:
#!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 9.3 MB/s eta 0:00:00


### Import packages

Import the necessary packages.

In [4]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
# Used to securely store your API key
from google.colab import userdata

In [5]:
import pandas as pd
import time
import random
import json
import numpy as np
import re

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [21]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('gemini-paid')

genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use `list_models` to see the available Gemini models:

* `gemini-pro`: optimized for text-only prompts.
* `gemini-pro-vision`: optimized for text-and-images prompts.

In [22]:
from google.colab import userdata
#userdata.get('gemini-paid')

In [23]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-exp-1114


Note: For detailed information about the available models, including their capabilities and rate limits, see [Gemini models](https://ai.google.dev/models/gemini). There are options for requesting [rate limit increases](https://ai.google.dev/docs/increase_quota). The rate limit for Gemini-Pro models is 60 requests per minute (RPM).

The `genai` package also supports the PaLM  family of models, but only the Gemini models support the generic, multimodal capabilities of the `generateContent` method.

## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model:

In [24]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
#model = genai.GenerativeModel('gemini-1.0-pro')

safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

The `generate_content` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. The available models only support text and images as input, and text as output.

In the simplest case, you can pass a prompt string to the <a href="https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content"><code>GenerativeModel.generate_content</code></a> method:

In [ ]:
#%%time
#response = model.generate_content("What is the meaning of life?")

CPU times: user 102 ms, sys: 10.7 ms, total: 113 ms
Wall time: 5.79 s


In [ ]:
# response.text

'As a large language model, I don\'t have feelings, beliefs, or personal experiences to draw upon, so I can\'t tell you the "meaning of life." That\'s a question humans have pondered for centuries, and there\'s no one definitive answer. \n\nHowever, I can offer some perspectives:\n\n* **It\'s up to you to decide:**  Many believe the meaning of life isn\'t inherent, but something you create through your actions, values, and experiences. What brings you joy? What impact do you want to have on the world?\n* **It\'s about the journey, not the destination:**  Some find meaning in the process of living, experiencing the world, learning, growing, and connecting with others. \n* **It\'s about contributing to something bigger:**  Others find meaning in dedicating themselves to a cause, helping others, or making the world a better place.\n* **It\'s about seeking knowledge and understanding:**  For some, the pursuit of knowledge, truth, and understanding the universe is a source of meaning.\n* **

In simple cases, the `response.text` accessor is all you need. To display formatted Markdown text, use the `to_markdown` function:

In [ ]:
#to_markdown(response.text)

> As a large language model, I can't tell you the meaning of life. That's a question philosophers and theologians have grappled with for centuries! 
> 
> The meaning of life is a personal and individual question. There's no one right answer.  It's up to each person to decide what gives their life meaning. 
> 
> Here are some things to consider:
> 
> * **Your values:** What is important to you? What do you believe in?
> * **Your purpose:** What do you want to accomplish in life? What impact do you want to make?
> * **Your experiences:** What brings you joy? What challenges you? What makes you feel fulfilled?
> 
> Ultimately, the meaning of life is what you make it. It's about finding what gives your life purpose and makes you feel fulfilled. 
> 
> If you're struggling with this question, it might be helpful to talk to a trusted friend, family member, or therapist. They can offer support and guidance as you explore what gives your life meaning. 


# Pre-process FinRAD_13K dataset

In [25]:
# read in csv
finRAD = pd.read_csv("data/FinRAD_13K.csv")
finRAD['terms'][1415]
# NOTE that the ô is properly encoded, but if you open the csv in excel, it will show up as √¥.
# Therefore, NOTE Don't open the csv in excel, it will mess up the encoding.!

'Itô’s Lemma'

In [28]:
# in Excel, we filter out all the non-English weird characters and manually fix them one by one.
# In A1 cell put:
# =IF(SUMPRODUCT(--(ISERR(FIND(MID(B1,ROW(INDIRECT("1:"&LEN(B1))),1),"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890!,.;:'/?!%&()-_+= "))))>0, "Non-English Character", "")

#finRAD['definitions'] = finRAD['definitions'].str.replace('â€™', "'")
#finRAD['definitions'] = finRAD['definitions'].str.replace('’', "'")
# .str.replace('@', "") is the KEY

# pattern = r'[^\w\sa-zA-Z0-9_\-:,. ]'
finRAD['definitions'] = finRAD['definitions'].str.replace('√¥', "o").str.replace('Äô', "'").str.replace('Äú', "").str.replace('Äù', "").str.replace('@', "").str.replace('Äì', "").str.replace('¬†', " ").str.replace('Äî', "").str.replace('√Ø', "i").str.replace('√©', "e")
# .apply(lambda x: re.sub(pattern, '', x))
finRAD['terms'] = finRAD['terms'].str.replace('√¥', "o").str.replace('Äô', "'").str.replace('Äú', "").str.replace('Äù', "").str.replace('@', "").str.replace('Äì', "").str.replace('¬†', " ").str.replace('Äî', "").str.replace('√Ø', "i").str.replace('√©', "e")


If the API failed to return a result, use `GenerateContentResponse.prompt_feedback` to see if it was blocked due to safety concerns regarding the prompt.

In [29]:
# construct definition_sents columns

finRAD['definition_sents'] = finRAD[['terms', 'definitions']].agg(': '.join, axis=1)
finRAD['definition_full_sents'] = finRAD[['terms', 'definitions']].agg(' means that '.join, axis=1)

In [30]:
def count_tokens(sentence):
    tokens = sentence.split()
    return len(tokens)

In [31]:
tokens_count = [count_tokens(sentence) for sentence in finRAD['definition_full_sents']]
print(max(tokens_count))
print(min(tokens_count))
print(np.mean(tokens_count))
print(np.median(tokens_count))

667
4
46.01418547895058
45.0


In [17]:
# Example tokens_count list
# tokens_count = [45, 60, 30, 55, 80, 50, 20, 65]

# Calculate the number of values greater than 50
count_greater_than_50 = sum(1 for token in tokens_count if token > 100)
# count_greater_than_50 = sum(1 for token in tokens_count if token > 50)

# Calculate the percentage
percentage_greater_than_50 = (count_greater_than_50 / len(tokens_count)) * 100

# Print the result
print(f"Percentage of token count values > 50: {percentage_greater_than_50:.2f}%")


Percentage of token count values > 50: 2.62%


In [19]:
print(len(tokens_count) - count_greater_than_50)

12769


In [32]:
len(finRAD['definition_full_sents'])

13112

In [ ]:
# try the first 1000 items
finRAD['summary_gemini-1.0-pro'] = finRAD['terms']
count = 0

for i,full_sent in enumerate(finRAD['definition_full_sents'][: 1000]):
  response = model.generate_content(full_sent + "Please summarize the above texts into one sentence only with shorter length.", safety_settings=safety_settings).text
  count = i
  if count % 100 == 0:
    print(count)
  finRAD.loc[i, 'summary_gemini-1.0-pro'] = response.replace('\n', '')

# finRAD.to_csv("dataset/FinRAD_13K_preprocessed.csv", index=False)
finRAD.to_csv("data/FinRAD_13K_shorter_summary.csv", index=False)


In [ ]:
finRAD_process = pd.read_csv("data/FinRAD_13K_preprocessed.csv")
finRAD_process['summary_gemini-1.0-pro'][1001]

'Multiplier'

In [ ]:
count = 999

In [ ]:
# NOTE Gemini API might get InternalError, so we have to store the results for every 1000 counts
for full_sent in finRAD_process['definition_full_sents'][1000: ]:
  count = count + 1
  if count % 100 == 0:
    print(count)
  response = model.generate_content(full_sent + "Please summarize the above contexts into one sentence only.", safety_settings=safety_settings).text
  finRAD_process.loc[count, 'summary_gemini-1.0-pro'] = response.replace('\n', '')

  if count % 1000 == 0:
    time.sleep(random.randint(11, 21))
    finRAD_process.to_csv("data/FinRAD_13K_preprocessed.csv", index=False)


In [ ]:
# finRAD['summary_gemini-1.0-pro'][998]
finRAD_process['summary_gemini-1.0-pro'][13111]

'ZOMBIE refers to a technically insolvent company that continues operating during the restructuring or reorganization process.'

In [ ]:
count

13111

In [ ]:
finRAD_process.to_csv("data/FinRAD_13K_gemini_summary.csv", index=False)

In [ ]:
 # time.sleep(random.randint(11, 30)) # force python to wait 10 sec before you execute the code
#full_sent = finRAD['definition_full_sents'][]
#gemini_summary.append(model.generate_content(full_sent + "Please summarize the above contexts into one sentence only.").text.replace('\n', ''))

In [ ]:
### 0.75-0.15-0.10 train-validation-test split.
import pandas as pd
import random

# Load the CSV file
file_path = 'data/FinRAD_13K_gemini_summary.csv'
data = pd.read_csv(file_path)

# Filter rows based on token length in the 'definition_full_sents' column
filtered_data = data[data['definition_full_sents'].str.replace('√¥', "o").str.replace('Äô', "'").str.replace('Äú', "").str.replace('Äù', "").str.replace('@', "").str.replace('Äì', "").str.replace('¬†', " ").str.replace('Äî', "").str.replace('√Ø', "i").str.replace('√©', "e").apply(lambda x: len(str(x).split()) <= 100)]

# Randomly select 12,000 examples
sampled_data = filtered_data.sample(n=12000, random_state=42)

# Split the data into train, validation, and test sets
train_data = sampled_data[:9000]
validation_data = sampled_data[9000:10800]
test_data = sampled_data[10800:]

# Write the data to text files
def save_to_txt(file_name, dataset):
    with open(file_name, 'w') as f:
        for sentence in dataset['definition_full_sents']:
            f.write(str(sentence).strip() + '\n')

save_to_txt('data/train_set.txt', train_data)
save_to_txt('data/validation_set.txt', validation_data)
save_to_txt('data/test_set.txt', test_data)

